In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/sample_data/colombian_elections.csv')

In [ ]:
df = df[['tweetText','polarity_value']].rename(columns={"tweetText":"texto","polarity_value":"sentimiento"})

In [ ]:
df.head()

In [ ]:
df['sentimiento'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(df.texto,df.sentimiento,random_state = 41)

In [ ]:
X_train

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [ ]:
regression = LinearRegression()
regression.fit(X_train,y_train)
acc = regression.score(X_test,y_test)
print(acc)

In [ ]:
df[2:3]

In [ ]:
df['sentiment'] = df['sentimiento'].replace(["N","P"],[0,1])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
texto = ["Las propuestas son buenas", "Las propuestas son malas"]

vectorizer = CountVectorizer(lowercase=False)

bag_of_words = vectorizer.fit_transform(texto)

In [ ]:
print(bag_of_words.shape)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
matrix = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns = vectorizer.get_feature_names())

In [ ]:
matrix

In [ ]:
vectorizer2 = CountVectorizer(lowercase=False, max_features = 100)

bag_of_words2 = vectorizer2.fit_transform(df.texto)

In [ ]:
print(bag_of_words2.shape)

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(bag_of_words2,df.sentiment,random_state = 41)

In [ ]:
regression = LinearRegression()
regression.fit(X_train,y_train)
acc = regression.score(X_test,y_test)
print(acc)

In [ ]:
regression = LogisticRegression()
regression.fit(X_train,y_train)
acc = regression.score(X_test,y_test)
print(acc)

In [ ]:
def clasificador(texto, columna_texto, categoria):
  vectorizer = CountVectorizer(lowercase=False, max_features = 400)
  bag_of_words = vectorizer.fit_transform(texto[columna_texto])
  X_train, X_test, y_train,y_test = train_test_split(bag_of_words,texto[categoria],random_state = 41)
  regression = LogisticRegression()
  regression.fit(X_train,y_train)
  return regression.score(X_test,y_test)

In [ ]:
clasificador(df, "texto", "sentiment")

In [ ]:
from wordcloud import WordCloud

In [ ]:
palabras = ' '.join([palabras for palabras in df['texto']])

In [ ]:
palabras

In [ ]:
len(palabras)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

def plot_cloud(wordcloud):
  plt.figure(figsize=(10,5))
  plt.imshow(wordcloud, interpolation = 'bilinear')
  plt.axis('off')

In [ ]:
wordcloud = WordCloud(width=800, height=500, collocations=False).generate(palabras)
plot_cloud(wordcloud)

In [ ]:
df_pos = df.query("sentimiento == 'P'")

In [ ]:
df_pos.head()

In [ ]:
def plot_cloud_neg(texto):
  df_neg = df.query("sentimiento == 'N'")
  palabras = ' '.join([palabras for palabras in df_neg['texto']])
  wordcloud = WordCloud(width=800, height=500, collocations=False).generate(palabras)
  plt.figure(figsize=(10,5))
  plt.imshow(wordcloud, interpolation = 'bilinear')
  plt.axis('off')

In [ ]:
def plot_cloud_pos(texto):
  df_pos = df.query("sentimiento == 'P'")
  palabras = ' '.join([palabras for palabras in df_pos['texto']])
  wordcloud = WordCloud(width=800, height=500, collocations=False).generate(palabras)
  plt.figure(figsize=(10,5))
  plt.imshow(wordcloud, interpolation = 'bilinear')
  plt.axis('off')

In [ ]:
plot_cloud_neg(df)

In [ ]:
plot_cloud_pos(df)

In [ ]:
import nltk 

frase =['el candidato es bueno', 'el candidato es malo']

frequencia = nltk.FreqDist(frase)

In [ ]:
frequencia

In [ ]:
from nltk import tokenize

In [ ]:
frase2 = 'el candidato quiere aprender LNP!'

token_espacio = tokenize.WhitespaceTokenizer()

frase_token = token_espacio.tokenize(frase2)

In [ ]:
print(frase_token)

In [ ]:
frase_token = token_espacio.tokenize(palabras)

frequencia = nltk.FreqDist(frase_token)

df_frequencia = pd.DataFrame({'palabra':list(frequencia.keys()), 'frequencia':list(frequencia.values())})

In [ ]:
df_frequencia.head()

In [ ]:
def histo_barras(texto, columna_texto, cantidad):  
  
  plt.figure(figsize=(12,8))

  palabras = ' '.join([palabras for palabras in df['texto']])

  frase_token = token_espacio.tokenize(palabras)

  frequencia = nltk.FreqDist(frase_token)

  df_frequencia = pd.DataFrame({'palabra':list(frequencia.keys()), 'frequencia':list(frequencia.values())})   

  df_frequencia = df_frequencia.nlargest(columns='frequencia', n = cantidad)

  ax = sns.barplot(data=df_frequencia, x='palabra', y='frequencia', color='gray')

  ax.set(ylabel = 'conteo')

  plt.show()

In [ ]:
histo_barras(df, "texto", 30)

In [ ]:
from nltk.corpus import stopwords

In [ ]:
irrelevantes = nltk.corpus.stopwords.words('spanish')

frase_process = list()

for opinion in df.texto:
  nueva_frase = list()
  palabras_texto = token_espacio.tokenize(opinion)
  for palabra in palabras_texto:
    if palabra not in irrelevantes:
      nueva_frase.append(palabra)
  frase_process.append(' '.join(nueva_frase))

df['tratamiento_1'] = frase_process  

In [ ]:
df.head()

In [ ]:
clasificador(df, "tratamiento_1", "sentiment")